<a href="https://colab.research.google.com/github/damerei/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/LS_DS_Unit_4_Sprint_Challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2*

# Sprint Challenge - Neural Network Foundations

Table of Problems

1. [Defining Neural Networks](#Q1)
2. [Perceptron on XOR Gates](#Q2)
3. [Multilayer Perceptron](#Q3)
4. [Keras MMP](#Q4)

<a id="Q1"></a>
## 1. Define the following terms:

- **Neuron:** A neuron - perhaps more accurately an artificial neuron - is a conceptual descriptor for a node in the mathematical artifice we call a neural network. 
- **Input Layer:** The input layer is the set of neurons representing the input data. For example, in the typical case of an image, a natural representation is to treat each pixel as an input corresponding to a neuron.
- **Hidden Layer:** The hidden layers are one or more layers of artificial neurons which represent various compressions, encodings, or otherwise transformations of the input layer, generally according to an activation function.
- **Output Layer:** The output layer represents as artificial neurons the results of the transformations encoded in the artificial neural network. 
- **Activation:** The activation function of a node (aka artificial neuron) is what transforms the input into an output. In the simplest feedforward network, you have an input layer which is transformed through a single application of the activation function in a single hidden layer. Typically you want to choose a function that is continuous (because differentiable and therefore restricted to giving appropriately small output variation in response to input variation) and monotonic (so that input-output variation is consistent in parity). Typical examples are sigmoid or arctan functions. 
- **Backpropagation:** Backpropagation is an algorithm - closely related to traditional numerical approximation by gradient methods such as the Gauss-Newton algorithm - to allow for progressive, recursive-iterative, and self-engineered improvement of a neural network's modeling effectiveness. Essentially one is constantly searching for the gradient of the error function (which in turn consists of the partial derivatives of the error function with respect to the weights), since this represents the rate of greatest change. Intuitively, by moving in that direction, one hypothetically reduces the error function at the greatest rate. Repeated application does not guarantee locating a global minimum, but certainly is helpful, and a number of techniques such as boosting and momentum exist to help prevent being caught in local minima. 

## 2. Perceptron on XOR Gates <a id="Q3=2"></a>

Create a perceptron class that can model the behavior of an AND gate. You can use the following table as your training data:

|x1	|x2|x3|	y|
|---|---|---|---|
1|	1|	1|	1|
1|	0|	1|	0|
0|	1|	1|	0|
0|	0|	1|	0|

In [0]:
import numpy as np

# Establish Inputs
inputs = np.array([
    [1,1,1],
    [1,0,1],
    [0,1,1],
    [0,0,1]
])

# Establish Target 
target = [[1], 
          [0], 
          [0], 
          [0]]

# Sigmoid functions
def sigmoid(x):
    return 1 / (1+np.exp(-x))

def sigmoid_derivative(x):
    sx = sigmoid(x)
    return sx* (1-sx)

In [0]:
weights = np.random.random((3,1)) - 1

weighted_sum = np.dot(inputs, weights)

activated_output = sigmoid(weighted_sum)

error = target - activated_output

adjustments = error * sigmoid_derivative(activated_output)

weights += np.dot(inputs.T, adjustments)

In [3]:
weights = np.random.random((3,1)) - 1

for iteration in range(10000):
    
    # Weighted Sum of inputs/weights
    
    weighted_sum = np.dot(inputs, weights)
    
    # Activate!
    activated_output = sigmoid(weighted_sum)
    
    # Calculate the Error
    error = target - activated_output
    
    # Adjustments
    adjustments = error * sigmoid_derivative(activated_output)
    
    # New weights. 
    weights += np.dot(inputs.T, adjustments)
    
print('Weights after Training')
print(weights)

print('Outputs after training')
print(activated_output)

Weights after Training
[[ 11.840066  ]
 [ 11.840066  ]
 [-18.04840824]]
Outputs after training
[[9.96430036e-01]
 [2.00873011e-03]
 [2.00873011e-03]
 [1.45146560e-08]]


These outputs are substantially the target [1, 0, 0, 0], differing in most cases by a few orders of magnitude. 

## 3. Multilayer Perceptron <a id="Q3"></a>

Implement a Neural Network Multilayer Perceptron class that uses backpropagation to update the network's weights.
Your network must have one hidden layer.
You do not have to update weights via gradient descent. You can use something like the derivative of the sigmoid function to update weights.
Train your model on the Heart Disease dataset from UCI:



In [0]:
class Neural_Network(object):
    def __init__(self,
                inputLayerSize=4,
                outputLayerSize=1,
                hiddenLayerSize=4):        
        #Define Hyperparameters
        self.inputLayerSize = inputLayerSize
        self.outputLayerSize = outputLayerSize
        self.hiddenLayerSize = hiddenLayerSize
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)  
        
        return dJdW1, dJdW2
    
    #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [0]:
from scipy import optimize
class trainer(object):
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))   
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []
        
        params0 = self.N.getParams()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

## 4. Keras MMP <a id="Q4"></a>

Implement a Multilayer Perceptron architecture of your choosing using the Keras library. Train your model and report its baseline accuracy. Then hyperparameter tune at least two parameters and report your model's accuracy.
Use the Heart Disease Dataset (binary classification)
Use an appropriate loss function for a binary classification task
Use an appropriate activation function on the final layer of your network.
Train your model using verbose output for ease of grading.
Use GridSearchCV to hyperparameter tune your model. (for at least two hyperparameters)
When hyperparameter tuning, show you work by adding code cells for each new experiment.
Report the accuracy for each combination of hyperparameters as you test them so that we can easily see which resulted in the highest accuracy.
You must hyperparameter tune at least 5 parameters in order to get a 3 on this section.